# Covid19 County-Level EDA Notebook
---
> ### [County-Level EDA Notebook](https://github.com/speediedan/covid19/blob/master/covid19_county_level_EDA.ipynb)
> * **This notebook can be used for manual EDA of county-level hotspot data**

> ### [The "Real-Time" County-Level Dashboard](county_covid_explorer.html):
> * **A "real-time"<sup>[1](#daily-onset-estimation)</sup> county-level dashboard w/ a focus on estimated effective reproduction number (R<sub>t</sub>)<sup>[2](#effective-reproduction-number-estimation)</sup>, 2nd order growth rates and confirmed infection density for most US counties (counties w/ > 0.03% confirmed infection density)**

> ### [The "Real-Time" Choropleth Dashboard](choropleth_covid_county_explorer.html):
> * **State and national choropleths for exploring the geographic distribution of "real-time"<sup>[1](#daily-onset-estimation)</sup> county-level R<sub>t</sub><sup>[2](#effective-reproduction-number-estimation)</sup> along with other relevant epidemiological statistics. Due to resource constraints, the national choropleth represents exclusively R<sub>t</sub> data while the state choropleths include additional county-level metrics. The national choropleth can currently be temporally evolved over a 14-day horizon.**



### Daily Onset Estimation
* It's important to be clear that these county-level R<sub>t</sub> estimates are "real-time" in the sense that the approach outlined in [(Bettencourt & Ribeiro, 2008)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0002185) is used while convolving the latest [onset-confirmed latency distribution](https://github.com/beoutbreakprepared/nCoV2019/tree/master/latest_data) onto daily reported cases (then adjusting for right-censoring) to obtain the estimated daily onset values. The latency between case onset and confirmation/reporting means that significant changes in local conditions still require some time (days) to be fully reflected in the R<sub>t</sub> estimates, but the estimate for a given point in time should improve with each passing day to a degree roughly correlated with the aforementioned onset-delay distribution.

### Effective Reproduction Number Estimation
   * I've extended [this great notebook](https://github.com/k-sys/covid-19/blob/master/Realtime%20R0.ipynb) to a county-level. 
   * Importantly, it should be noted that (as of 2020.05.12) access to testing is continuing to increase and test positivity rates are therefore changing at a [substantial rate](https://www.cdc.gov/coronavirus/2019-ncov/covid-data/covidview/index.html). As the testing bias continues to evolve in the near-term, one should recognize that point R<sub>t</sub> estimates will be biased to be higher than ground truth R<sub>t</sub>. There are approaches that can [mitigate this bias to a limited extent](http://freerangestats.info/blog/2020/05/09/covid-population-incidence) but fundamentally, we don't have sufficient data to eliminate the bias at this point so I've deprioritized making those model adjustments at the moment (I may make testing-related adjustments in the future though and PRs are welcome!). Fortunately, as testing access and bias stabilize at a level that increases validity of confirmed case counts, these R<sub>t</sub> estimates should become increasingly accurate. I think we can expect hotspot monitoring tools such as this to have utility for a number of months, so this initial period of testing volatility does not nullify their value.
   * The most salient change I've made in the process of the extension is that rather than using a prior of gamma-distributed generation intervals to estimate R (which seems totally reasonable), I'm experimenting with incorporating more locally-relevant information by calculating an R<sub>0</sub> using initial incidence data from each locality.
   * For execution environments that are compute-constrained, I've also provided (but left disabled) some performance enhancing functions that cut execution time by about 50% at the cost of ~5% accuracy.
   
### SEIR Model Notes
* #### At the time the SEIR model component of this notebook was written (2020.03.30) there remained significant uncertainty regarding some sars-cov-2 parameters. The data fit varied substantially by county so I used what I perceived (N.B.: w/ no personal epidemiological expertise!!) to be the consensus values, documented below:

| Parameter   | Source  | Reference Value     |
| :---        | :----:  |     ---:            |
| Latent Period   | [Lin et al., 2020](https://www.ijidonline.com/article/S1201-9712(20)30117-X/fulltext) | 3   |
| Latent Period   | [Wu et al., 2020](https://www.sciencedirect.com/science/article/pii/S0140673620302609) | 3     |
| Latent Period   | [Li et al., 2020](https://www.medrxiv.org/content/10.1101/2020.03.06.20031880v1.full.pdf) | 2 |
| Serial Interval | [Nishura et al. 2020](https://www.ijidonline.com/article/S1201-9712(20)30119-3/pdf) | 4.6 |
| Serial Interval | [Li et al., 2020](https://www.nejm.org/doi/pdf/10.1056/NEJMoa2001316?articleTools=true) | 7.5 |
| Incubation Period | [Li et al., 2020](https://www.nejm.org/doi/pdf/10.1056/NEJMoa2001316?articleTools=true) | 5.2 |
| Infectious Period | [Li et al., 2020](https://www.nejm.org/doi/pdf/10.1056/NEJMoa2001316?articleTools=true) | 2.3 |
| Infectious Period | [Zhou et al., 2020](https://www.medrxiv.org/content/10.1101/2020.02.24.20026773v1.full.pdf) | 6 |
| Infectious Period | [Bi et al., 2020](https://www.medrxiv.org/content/10.1101/2020.03.03.20028423v3) | 1.5
| Infectious Period | [Kucharski et al., 2020](https://cmmid.github.io/topics/covid19/current-patterns-transmission/wuhan-early-dynamics.html) | 2.9
| Time to Hospitalization | [Huang et al., 2020](https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(20)30183-5/fulltext) | 8
| Mean Hospitalization Period | [Wang et al., 2020](https://jamanetwork.com/journals/jama/fullarticle/2761044?guestAccessKey=f61bd430-07d8-4b86-a749-bec05bfffb65) | 12
| Hospitalization Rate | [Ferguson et al., 2020](https://spiral.imperial.ac.uk/bitstream/10044/1/77482/5/Imperial%20College%20COVID19%20NPI%20modelling%2016-03-2020.pdf) (weighted by us demo by [Covid Act Now](https://covidactnow.org/model)) | 0.073

In [12]:
import os
from IPython.core.debugger import set_trace
import pandas as pd
import c19_analysis.dataprep_utils as covid_utils
from c19_analysis.dataprep_flow import build_latest_case_data
import config
import c19_analysis.dataprep_utils as covid_utils
import c19_analysis.bayesian_rt_est as bayes_rt
# import c19_analysis.cust_seir_model as cust_seir
import dashboard.rt_explorer as rt_explorer
import dashboard.choropleth_explorer as choropleth_explorer
import dashboard.static_mpl_viz as static_mpl_viz

In [13]:
# Build/Update Core Case Data
usafacts_delta_df, updated = build_latest_case_data()

No update to case data source, loading core case data from cache


In [14]:
# Update Effective R Estimates
# Bayesian Rt Estimation
rt_df = bayes_rt.gen_rt_df(usafacts_delta_df)

No update to core case data, loaded Rt estimates from cache


In [15]:
# The applicable scope of this particular SEIR model was only the initial, exponential stage of the outbreak and remains here for reference only.
# cust_seir.gen_seir_viz(rt_df)

In [16]:
# Prepare dataframes necessary for downstream dashboard generation
# rt_df = core rt_explorer dataframe source
# viz_df_instances = used to generate temporal evolution of national choropleth
# status_df = rt_explorer table dataframe source
# county_date_instances: list of dates for which to generate national choropleth
rt_df, viz_df_instances, status_df, county_date_instances = covid_utils.prep_dashboard_dfs(rt_df)

In [7]:
# feel free to explore the data using the dataframes generated above
# e.g.:
# view 5 largest confirmed % infected
s = status_df.nlargest(5, 'confirmed %infected')
s = s.style.apply(covid_utils.color_mask, subset=['confirmed %infected'], thresh=0.9)
display(s)

,,,,,Estimated Onset Cases,Total Estimated Cases,node_start_dt,daily new cases ma,Confirmed New Cases,growth_rate,growth_period_n,growth_period_n-1,2nd_order_growth,Rt,90_CrI_LB,90_CrI_UB,confirmed %infected
id,estimated_pop,name,stateAbbr,Date,,,,,,,,,,,,,
47169,11012,"Trousdale County, TN",TN,2020-05-26 00:00:00,0,1405,2020-03-13 00:00:00,1,0,0,0.0007,0.000875,-0.2,0.4,0,1.23,12.76
31043,20083,"Dakota County, NE",NE,2020-05-26 00:00:00,27,1750,2020-03-14 00:00:00,20,27,0.0157,0.011925,0.01595,-0.2524,0.91,0.46,1.27,8.71
5079,13383,"Lincoln County, AR",AR,2020-05-26 00:00:00,0,987,2020-03-07 00:00:00,2,0,0,0.00205,0.001,1.05,0.77,0.05,1.44,7.38
27105,21924,"Nobles County, MN",MN,2020-05-26 00:00:00,9,1541,2020-03-13 00:00:00,11,9,0.0059,0.00725,0.00835,-0.1317,0.89,0.39,1.36,7.03
31037,10881,"Colfax County, NE",NE,2020-05-26 00:00:00,2,642,2020-03-24 00:00:00,5,2,0.0031,0.00795,0.016775,-0.5261,0.73,0.14,1.22,5.9
